#`mountUmount(`<font size="3px" color="#01c968">`Gdrive`</font>`)`



In [1]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive!
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive


#`Setup And Update ComfyUI`



In [2]:
from pathlib import Path

OPTIONS = {}

DRIVE_PATH = ""  # @param {type:"string"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = '/content/ComfyUI'
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if DRIVE_PATH:

    WORKSPACE = DRIVE_PATH+"/ComfyUI"
    %cd {DRIVE_PATH}

![ ! -d WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

%cd {WORKSPACE}

-= Initial setup ComfyUI =-
Cloning into 'ComfyUI'...
remote: Enumerating objects: 32927, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 32927 (delta 72), reused 26 (delta 26), pack-reused 32807 (from 3)
Receiving objects: 100% (32927/32927), 72.45 MiB | 22.74 MiB/s, done.
Resolving deltas: 100% (22451/22451), done.
/content/ComfyUI
-= Updating ComfyUI =-
Already up to date.
-= Install dependencies =-
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121, https://download.pytorch.org/whl/cu118, https://download.pytorch.org/whl/cu117
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.2/21.2 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 134.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15

# `Models Download`

## Download MODELS

In [31]:
%cd {WORKSPACE}
# Install Aria2
!apt-get -y install -qq aria2


from google.colab import userdata

CIVITAI_API_TOKEN = userdata.get('CIVITAI_API_TOKEN')
print("Loaded API key:", "✅" if CIVITAI_API_TOKEN else "❌ Not found")

# Install custom nodes

def install_custom_node(url):
  %cd /content/ComfyUI/custom_nodes
  !git clone {url}

def downloadModel(url, filename = None):
  if 'huggingface.co' in url:
    if filename is None:
      filename = url.split('/')[-1]
      filename = filename.removesuffix('?download=true')
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  else:
    # civitai
    if filename:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}?token={CIVITAI_API_TOKEN} -o {filename}
    else:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}?token={CIVITAI_API_TOKEN}


# install custom nodes
#install_custom_node('https://github.com/ltdrdata/ComfyUI-Manager.git')
#install_custom_node('https://github.com/ltdrdata/ComfyUI-Impact-Pack')


# download models
%cd ./models/checkpoints

# CyberRealistic Pony
# downloadModel('https://civitai.com/api/download/models/2514310')
# downloadModel('https://civitai.com/api/download/models/2167369')
# downloadModel('https://civitai.com/api/download/models/2071650')
# downloadModel('https://civitai.com/api/download/models/1767402')
# downloadModel('https://civitai.com/api/download/models/1071060')
# %cd ..
# %cd loras
# downloadModel('https://civitai.com/api/download/models/448977')
# downloadModel('https://civitai.com/api/download/models/1071060')
# downloadModel('https://civitai.com/api/download/models/382152')
# downloadModel('https://civitai.com/api/download/models/925417')
# downloadModel('https://civitai.com/api/download/models/1573142')
# downloadModel('https://civitai.com/api/download/models/1748207')
# downloadModel('https://civitai.com/api/download/models/1965505')
%cd ..
%cd upscale_models
downloadModel('https://civitai.com/api/download/models/658202')

%cd ..
%cd controlnet
downloadModel('https://civitai.com/api/download/models/125843')
# Protovision XL
#downloadModel('https://civitai.com/api/download/models/201514')
# Juggernaut XL Hyper
#downloadModel('https://civitai.com/api/download/models/471120')
# Juppernaut X
#downloadModel('https://civitai.com/api/download/models/456194')
# Juggernaut v8
#downloadModel('https://civitai.com/api/download/models/288982')

# Huggingface example
#downloadModel('https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors?download=true')

%cd {WORKSPACE}

/content/ComfyUI
Loaded API key: ✅
/content/ComfyUI/models/checkpoints
/content/ComfyUI/models
/content/ComfyUI/models/upscale_models

02/22 01:01:42 [ERROR] CUID#7 - Download aborted. URI=https://civitai.com/api/download/models/658202?token=67697df1871dcc1146a72e10d0ff40c0
Exception: [AbstractCommand.cc:351] errorCode=22 URI=https://civitai.com/api/download/models/658202?token=67697df1871dcc1146a72e10d0ff40c0
  -> [HttpSkipResponseCommand.cc:239] errorCode=22 The response status is not successful. status=500

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
8b99cc|ERR |       0B/s|https://civitai.com/api/download/models/658202?token=67697df1871dcc1146a72e10d0ff40c0

Status Legend:
(ERR):error occurred.

aria2 will resume download if the transfer is restarted.
If there are any errors, then see the log file. See '-l' option in help/man page for details.
/content/ComfyUI/models
/content/ComfyUI/models/contr

UI

In [25]:
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git ./custom_nodes/ComfyUI-Manager
!git clone https://github.com/weilin9999/WeiLin-Comfyui-Tools.git ./custom_nodes/ComfyUI-Manager

fatal: destination path './custom_nodes/ComfyUI-Manager' already exists and is not an empty directory.
Cloning into 'WeiLin-Comfyui-Tools'...
remote: Enumerating objects: 2230, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 2230 (delta 88), reused 74 (delta 74), pack-reused 2085 (from 2)
Receiving objects: 100% (2230/2230), 17.27 MiB | 19.73 MiB/s, done.
Resolving deltas: 100% (1390/1390), done.


## OTHERS

In [15]:
# SD1.5
!wget -c https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -P ./models/checkpoints/

# Some SD1.5 anime style
!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors -P ./models/checkpoints/

# VAE
!wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P ./models/vae/

--2026-02-21 22:20:03--  https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt
Resolving huggingface.co (huggingface.co)... 18.164.174.118, 18.164.174.23, 18.164.174.17, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.118|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /stable-diffusion-v1-5/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt [following]
--2026-02-21 22:20:03--  https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cas-bridge.xethub.hf.co/xet-bridge-us/66d19580e2632490a6bc5829/773d9e5ddbf6a84612066b3ea00a0bcfe4ba870a9befc88816f71b5927e76e38?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Content-Sha256=UNSIGNED-PAYLOAD&X-Amz-Credential=cas%2F20260221%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20260221T222003Z&

## LIST MODELS

In [14]:
!ls -al ./models/checkpoints/

total 23212612
drwxr-xr-x  2 root root       4096 Feb 21 22:10 .
drwxr-xr-x 23 root root       4096 Feb 21 21:48 ..
-rw-r--r--  1 root root 5570803627 Feb 21 21:53 AbyssOrangeMix2_hard.safetensors
-rw-r--r--  1 root root 6938040682 Feb 21 21:52 cyberrealisticPony_v127Alt.safetensors
-rw-r--r--  1 root root          0 Feb 21 21:48 put_checkpoints_here
-rw-r--r--  1 root root   57417531 Feb 21 22:10 Super_Eye_Detailer_By_Stable_Yogi_SDPD0.safetensors
-rw-r--r--  1 root root 4265380512 Feb 21 21:53 v1-5-pruned-emaonly.ckpt
-rw-r--r--  1 root root 6938040714 Feb 21 22:10 waiANIPONYXL_v140.safetensors


# `START ComfyUI  & Expose Server (MANUAL)`

## Download Prerequisits

In [6]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

--2026-02-21 21:54:00--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2026.2.0/cloudflared-linux-amd64.deb [following]
--2026-02-21 21:54:00--  https://github.com/cloudflare/cloudflared/releases/download/2026.2.0/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/4fddf4d7-e02d-44dc-9e5a-ef9e28afdd54?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-02-21T22%3A35%3A52Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64.deb&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&

In [8]:
!python main.py --dont-print-server

Found comfy_kitchen backend triton: {'available': True, 'disabled': True, 'unavailable_reason': None, 'capabilities': ['apply_rope', 'apply_rope1', 'dequantize_nvfp4', 'dequantize_per_tensor_fp8', 'quantize_nvfp4', 'quantize_per_tensor_fp8']}
Found comfy_kitchen backend cuda: {'available': True, 'disabled': True, 'unavailable_reason': None, 'capabilities': ['apply_rope', 'apply_rope1', 'dequantize_nvfp4', 'dequantize_per_tensor_fp8', 'quantize_nvfp4', 'quantize_per_tensor_fp8', 'scaled_mm_nvfp4']}
Found comfy_kitchen backend eager: {'available': True, 'disabled': False, 'unavailable_reason': None, 'capabilities': ['apply_rope', 'apply_rope1', 'dequantize_nvfp4', 'dequantize_per_tensor_fp8', 'quantize_nvfp4', 'quantize_per_tensor_fp8', 'scaled_mm_nvfp4']}
Checkpoint files will always be loaded safely.
Total VRAM 14913 MB, total RAM 12976 MB
pytorch version: 2.10.0+cu128
xformers version: 0.0.35
Set vram state to: NORMAL_VRAM
Device: cuda:0 Tesla T4 : cudaMallocAsync
Using async weight o

# `START ComfyUI & Expose Server`

## Download Prerequisits

In [10]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

--2026-02-21 21:56:53--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2026.2.0/cloudflared-linux-amd64.deb [following]
--2026-02-21 21:56:54--  https://github.com/cloudflare/cloudflared/releases/download/2026.2.0/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/4fddf4d7-e02d-44dc-9e5a-ef9e28afdd54?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-02-21T22%3A37%3A20Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64.deb&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&

## CF Tunnel

In [32]:
import os

# Ensure we are in the ComfyUI directory
%cd {WORKSPACE}

# Run main.py with --help to display all parameters
!python main.py --help

/content/ComfyUI
usage: main.py [-h] [--listen [IP]] [--port PORT] [--tls-keyfile TLS_KEYFILE]
               [--tls-certfile TLS_CERTFILE] [--enable-cors-header [ORIGIN]]
               [--max-upload-size MAX_UPLOAD_SIZE]
               [--base-directory BASE_DIRECTORY]
               [--extra-model-paths-config PATH [PATH ...]]
               [--output-directory OUTPUT_DIRECTORY]
               [--temp-directory TEMP_DIRECTORY]
               [--input-directory INPUT_DIRECTORY] [--auto-launch]
               [--disable-auto-launch] [--cuda-device DEVICE_ID]
               [--default-device DEFAULT_DEVICE_ID]
               [--cuda-malloc | --disable-cuda-malloc]
               [--force-fp32 | --force-fp16]
               [--fp32-unet | --fp64-unet | --bf16-unet | --fp16-unet | --fp8_e4m3fn-unet | --fp8_e5m2-unet | --fp8_e8m0fnu-unet]
               [--fp16-vae | --fp32-vae | --bf16-vae] [--cpu-vae]
               [--fp8_e4m3fn-text-enc | --fp8_e5m2-text-enc | --fp16-text-enc | --fp32

In [ ]:
import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2026-02-22 01:05:03.059
** Platform: Linux
** Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
** Python executable: /usr/bin/python3
** ComfyUI Path: /content/ComfyUI
** ComfyUI Base Folder Path: /content/ComfyUI
** User directory: /content/ComfyUI/user
** ComfyUI-Manager config path: /content/ComfyUI/user/__manager/config.ini
** Log path: /content/ComfyUI/user/comfyui.log

Prestartup times for custom nodes:
   7.1 seconds: /content/ComfyUI/custom_nodes/ComfyUI-Manager

Found comfy_kitchen backend eager: {'available': True, 'disabled': False, 'unavailable_reason': None, 'capabilities': ['apply_rope', 'apply_rope1', 'dequantize_nvfp4', 'dequantize_per_tensor_fp8', 'quantize_nvfp4', 'quantize_per_tensor_fp8', 'scaled_mm_nvfp4']}
Found comfy_kitchen backend triton: {'available': True, 'disabled': True, 'unavailable_reason': None, 'capabilities': ['apply_

## localtunnel

In [18]:
# localtunnel
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()
%cd /content/ComfyUI
!python main.py --dont-print-server

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇
changed 22 packages in 2s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇/content/ComfyUI
Found comfy_kitchen backend eager: {'available': True, 'disabled': False, 'unavailable_reason': None, 'capabilities': ['apply_rope', 'apply_rope1', 'dequantize_nvfp4', 'dequantize_per_tensor_fp8', 'quantize_nvfp4', 'quantize_per_tensor_fp8', 'scaled_mm_nvfp4']}
Found comfy_kitchen backend triton: {'available': True, 'disabled': True, 'unavailable_reason': None, 'capabilities': ['apply_rope', 'apply_rope1', 'dequantize_nvfp4', 'dequantize_per_tensor_fp8', 'quantize_nvfp4', 'quantize_per_tensor_fp8']}
Found comfy_kitchen backend cuda: {'available': True, 'disabled': True, 'unavailable_reason': None, 'capabilities': ['apply_rope', 'apply_rope1', 'dequantize_nvfp4', 'dequantize_per_tensor_fp8', 'quantize_nvfp4', 'quantize_per_tensor_fp8', 'scaled_mm_nvfp4']}
Checkpoint files will always be loaded safely.
Total VRAM 14913 MB, total RAM 12976 MB
pytorch version: 